In [1]:
import os
import cv2
from PIL import Image
import numpy as np
import pickle as pkl
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from processer import processer, extractGreen, ACE, equalize
from dataset import PlantSeedDataset

In [ ]:
datasets_path = "..\\dataset\\classifer\\train"
    
transform = transforms.Compose([
        transforms.Resize((256, 256)),
        np.asarray,
        transforms.Lambda(lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2BGR))
        ])
dataset = PlantSeedDataset(datasets_path, transform=transform)
dataloader = DataLoader(dataset)

In [2]:
for img, label, img_name in dataset:
    print(type(img), type(label), type(img_name))
    break

<class 'numpy.ndarray'> <class 'str'> <class 'str'>


In [41]:
i = 0
for img, label, img_name in dataset:
    # if i != 249:
    #     i += 1
    #     continue
    # goushi = img
    gre_img = cv2.medianBlur(extractGreen(ACE(img)), 5)
    # gre_img = extractGreen(img, enable_ACE=True)
    folder_name = f'..\\output\\ACEGreMB\\{label}'
    if not os.path.exists(folder_name):   
        os.makedirs(folder_name)   
    cv2.imwrite(folder_name + f'\\{img_name}', gre_img)
    # cv2.imshow("des", gre_img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # break

In [54]:
# 把ACE后再抽绿的保存到跟直接抽绿的一个文件夹下
data_dir = "..\\output\\ACEGreMB"
for root, dirs, _ in os.walk(data_dir):
    for sub_dir in dirs:
        # print(12316561)
        img_names = os.listdir(os.path.join(root, sub_dir))
        img_names = list(filter(lambda x: x.endswith('.png'), img_names))
        for i in range(len(img_names)):
            img_name = img_names[i]
            path_img = os.path.join(root, sub_dir, img_name)
            save_path = f'..\\output\\fortrain\\{sub_dir}'
            # print(i)
            cv2.imwrite(save_path + f'\\{img_name}'[:-4] + '_ace.png', cv2.imread(path_img))

In [49]:
testdatasets_path = "..\\dataset\\classifer\\test"
    
transform = transforms.Compose([
        transforms.Resize((256, 256)),
        np.asarray,
        transforms.Lambda(lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2BGR))
        ])
test_dataset = PlantSeedDataset(testdatasets_path, transform=transform)
test_dataloader = DataLoader(test_dataset)
for img, label, img_name in test_dataset:
    gre_img = cv2.medianBlur(extractGreen(ACE(img)), 5)
    folder_name = f'..\\output\\test\\{label}'
    if not os.path.exists(folder_name):   
        os.makedirs(folder_name)   
    cv2.imwrite(folder_name + f'\\{img_name}'[:-4]+'_ace.png', gre_img)
for img, label, img_name in test_dataset:
    gre_img = extractGreen(img)
    folder_name = f'..\\output\\test\\{label}'
    if not os.path.exists(folder_name):   
        os.makedirs(folder_name)   
    cv2.imwrite(folder_name + f'\\{img_name}', gre_img)

直接抽绿色，有些较暗的图抽不到主干，先ACE后抽有些图会引入过多噪声，将他俩合并，就当数据增强了，一块训练

In [5]:
cv2.imshow("des", extractGreen(dataset[249][0], enable_ACE=True))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
from processer import ACE, equalize
def extractGreen(image, enable_ACE=False, ratio=4, radius=3):  # kernel_size 滤波卷积核大小
    # 绿色范围
    lower_green = np.array([30, 43, 46], dtype="uint8")  # 颜色下限[色调，饱和度，亮度]
    upper_green = np.array([85, 255, 255], dtype="uint8")  # 颜色上限
    
    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # 根据阈值找到对应颜色，二值化
    mask = cv2.inRange(img_hsv, lower_green, upper_green)
    # 按位与
    output = cv2.bitwise_and(image, image, mask=mask)
    if enable_ACE:
        ace_img = ACE(output, ratio, radius)
        output = cv2.bitwise_and(ace_img, ace_img, mask=mask)
    return output

cv2.imshow("des", extractGreen(dataset[249][0]))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
acedatasets_path = "..\\output\\ACE43"
    
transform = transforms.Compose([
        transforms.Resize((256, 256)),
        np.asarray,
        transforms.Lambda(lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2BGR))
        ])
acedataset = PlantSeedDataset(acedatasets_path, transform=transform)
acedataloader = DataLoader(acedataset)

In [7]:
for img, label, img_name in acedataset:
    cv2.imshow("des", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    break

In [37]:
from processer import ACE, equalize
hard_img = '..\\dataset\\classifer\\train\\Common Chickweed\\dzcivxzgey.png'
hard_img = cv2.resize(cv2.imread(hard_img), (256, 256))
gre = extractGreen(hard_img)
equgre = extractGreen(equalize(hard_img))
acegre = extractGreen(ACE(hard_img))
img_blur = cv2.medianBlur(acegre, 5)  # 中值滤波
cv2.imshow("hard_img", hard_img)
cv2.imshow("gre", gre)
cv2.imshow("equgre", equgre)
cv2.imshow("acegre", acegre)
cv2.imshow("img_blur", img_blur)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
train_path = "..\\output\\fortrain"
test_path = "..\\output\\test"
transform = transforms.Compose([
        transforms.Resize((256, 256)),
        np.asarray,
        transforms.Lambda(lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2BGR))
        ])
traindataset = PlantSeedDataset(train_path, transform=transform)
testdataset = PlantSeedDataset(test_path, transform=transform)
# cv2.imshow("train", traindataset[1][0])
# cv2.imshow("test", testdataset[1][0])
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [58]:
import pickle as pkl
from processer import SIFT

sift_features = []
for img, label, img_name in traindataset:
    kp, sift_feature = SIFT(img)
    if type(sift_feature) == type(None):
        print(f'{label}, {img_name}!!!!!!!!!!!!!!!!!!!!')
        continue
    sift_features.append(sift_feature)
print('test!!!!!!!!!!!!!!!!!!!!')   
print(f'train_len:{len(sift_features)}') 
for img, label, img_name in testdataset:
    kp, sift_feature = SIFT(img)
    if type(sift_feature) == type(None):
        print(f'{label}, {img_name}!!!!!!!!!!!!!!!!!!!!')
        continue
    sift_features.append(sift_feature)

with open("train&test_sift_features.pkl", "wb") as f:  # 序列化
    pkl.dump(sift_features, f) 
len(sift_features)

Black-grass, lbvtxfiakc.png!!!!!!!!!!!!!!!!!!!!
Black-grass, wfihcvrygp_ace.png!!!!!!!!!!!!!!!!!!!!
Common Chickweed, dzcivxzgey.png!!!!!!!!!!!!!!!!!!!!
Loose Silky-bent, tevntasarh_ace.png!!!!!!!!!!!!!!!!!!!!
Loose Silky-bent, uptscykujw.png!!!!!!!!!!!!!!!!!!!!
Maize, xcbwqxdzuq.png!!!!!!!!!!!!!!!!!!!!
test!!!!!!!!!!!!!!!!!!!!
train_len:8872


11080

In [3]:
import pickle as pkl
with open('gre_img.pkl', 'rb') as f:
    gre_imgs = pkl.load(f)
with open('sift_features.pkl', 'rb') as f:
    sift_features = pkl.load(f)

In [2]:
import pickle as pkl
with open("train&test_sift_features.pkl", "rb") as f:  # 序列化
    sift_features = pkl.load(f) 
len

11080

In [3]:
from processer import bow_feature, bow_init
bow_extractor = bow_init(sift_features, 256)

NameError: name 'gre_imgs' is not defined

In [13]:
imgs = [x for x, _, _ in traindataset]
print(len(imgs))
test_img_list = [x for x, _, _ in testdataset]
imgs += test_img_list
print(len(imgs))

8878
11086


In [3]:
with open("voc_bow.pkl", "rb") as f:  # 序列化
    voc = pkl.load(f) 
# FLANN匹配  
flann_params = dict(algorithm=1,tree=5)
flann = cv2.FlannBasedMatcher(flann_params,{})

#初始化bow提取器(设置词汇字典),用于提取每一张图像的BOW特征描述
sift = cv2.SIFT_create()
bow_extractor = cv2.BOWImgDescriptorExtractor(sift, flann)        
bow_extractor.setVocabulary(voc)

In [5]:
from processer import bow_feature
all_feature_bow = bow_feature(bow_extractor, traindataset.imgs + testdataset.imgs)
print(len(all_feature_bow))
train_feature_bow = all_feature_bow[:8872]
test_feature_bow = all_feature_bow[8872:]
with open("train_feature_bow.pkl", "wb") as f:  # 序列化
    pkl.dump(train_feature_bow, f) 
with open("test_feature_bow.pkl", "wb") as f:  # 序列化
    pkl.dump(test_feature_bow, f) 
len(train_feature_bow), len(test_feature_bow)

11080


(8872, 2208)

In [6]:
from processer import HOG
hog_features = []
for img, label, img_name in traindataset:
    hog_feature, hog_img = HOG(img)
    folder_name = f'..\\output\\HOG\\{label}'
    if not os.path.exists(folder_name):   
        os.makedirs(folder_name)   
    cv2.imwrite(folder_name + f'\\{img_name}', hog_img)
    hog_features.append(hog_feature)
with open("train_hog.pkl", "wb") as f:  # 序列化
    pkl.dump(hog_features, f) 
len(hog_features)

In [5]:
len(all_feature_bow)

4440

In [7]:
all_feature_bow[0].shape

(1, 100)

In [6]:
import pickle as pkl
with open('gre_img.pkl', 'rb') as f:
    gre_imgs = pkl.load(f)
hog_features = []
for img in gre_imgs:
    hog_feature, hog_img = HOG(img)
    hog_features.append(hog_feature)
with open("hog_features.pkl", "wb") as f:  # 序列化
    pkl.dump(hog_features, f) 

In [29]:
with open('hog_features.pkl', 'rb') as f:
    hog_features = pkl.load(f)

In [2]:
hog_features[0].shape

(129600,)

In [28]:
feature_lbps = []
for img in gre_imgs:
    feature_lbp = LBP(img, n_points=2, radius=2)
    feature_lbps.append(feature_lbp)
with open("feature_lbps22.pkl", "wb") as f:  # 序列化  后两位为参数 不加的为默认参数
    pkl.dump(feature_lbps, f) 

In [4]:
# feature_lbps[0].shape
# cv2.imwrite("lbp.png", feature_lbps[0])
with open('gre_img.pkl', 'rb') as f:
    gre_imgs = pkl.load(f)

True

In [27]:
i = 265
org_img = gre_imgs[i]
cv2.imwrite(f"org_img_{i}.png", org_img)
lbp_img = LBP(org_img, n_points=2, radius=2, method='var')
cv2.imwrite(f"lbp_{i}.png", lbp_img)

True

In [37]:
i = 0
org_img = gre_imgs[i]
cv2.imwrite(f"org_img_{i}.png", org_img)
hog_feature, hog_img = HOG(org_img, orientations=9, pixels_per_cell=8, cells_per_block=3)
# hog_feature, hog_img = HOG(org_img)
cv2.imwrite(f"hog_{i}.png", hog_img)
hog_feature.shape

(72900,)

In [38]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=3)  # t分布随机邻居嵌入 降维
embeddings = tsne.fit_transform(np.asarray(hog_features[:200]))

OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

In [57]:
from sklearn.decomposition import KernelPCA
rbf_pca = KernelPCA(n_components = 100, kernel="rbf", gamma=0.04) 
hog_pca = rbf_pca.fit_transform(np.asarray(hog_features)) 

In [88]:
hog_pca.shape

(4440, 100)

In [89]:
np.asarray(hog_features).shape

(4440, 129600)

In [59]:
np.save('hog_pca.npy', hog_pca)

In [60]:
with open("feature_lbps22.pkl", "rb") as f:  # 序列化  后两位为参数 不加的为默认参数
    lbp_features = pkl.load(f)

In [69]:
for i in range(len(lbp_features)):
    lbp_features[i] = lbp_features[i].reshape(256*256, 3)

In [74]:
np.asarray(lbp_features).shape

(4440, 65536, 3)

In [92]:
for i in range(len(lbp_features)):
    lbp_features[i] = lbp_features[i].reshape(3, -1)

In [93]:
lbp_features[0].shape

(3, 65536)

In [73]:
np.transpose(lbp_features[0]).shape

(3, 65536)

In [94]:
import pandas as pd
lbp_fill = []
for lbp_feature in lbp_features:
    feature_df = pd.DataFrame(lbp_feature)
    lbp_fill.append(feature_df.fillna(0))  # 将缺失值部分填充0

In [95]:
lbp_fill[0].shape

(3, 65536)

In [97]:
lbp_pca = []
for lbp_feature in lbp_fill:
    rbf = KernelPCA(n_components = 1000, kernel="rbf", gamma=0.04)
    lbp_pca1 = rbf.fit_transform(lbp_feature)
    print(lbp_pca1.shape)
    break
    rbf_pca = KernelPCA(n_components = 100, kernel="rbf", gamma=0.04) 
    lbp_pca.append(rbf_pca.fit_transform(np.transpose(lbp_feature)))
# lbp_pca = np.asarray(lbp_pca)
# np.save('lbp_pca.npy', lbp_pca)

(3, 3)


In [98]:
lbp_fill_2d = np.asarray(lbp_fill).reshape(-1, 65536)

In [99]:
lbp_fill_2d.shape

(13320, 65536)

In [100]:
rbf = KernelPCA(n_components = 100, kernel="rbf", gamma=0.04)
lbp_pca = rbf.fit_transform(lbp_fill_2d)

In [101]:
lbp_pca.shape

(13320, 100)

In [103]:
lbp_pca = lbp_pca.reshape(-1, 3, 100)

In [104]:
lbp_pca.shape

(4440, 3, 100)

In [105]:
np.save('lbp_pca.npy', lbp_pca)

In [106]:
from xgboost import XGBClassifier
import xgboost
from sklearn.model_selection import train_test_split

In [2]:
import pickle as pkl
with open("all_feature_bow.pkl", "rb") as f:  # 序列化
    all_feature_bow = pkl.load(f) 

EOFError: Ran out of input